# Yahoo News Crawl

In [1]:
import requests
from bs4 import BeautifulSoup

req = requests.get('https://tw.news.yahoo.com/')
soup = BeautifulSoup(req.text, 'lxml')

### Get categories

In [2]:
categories = []
for category in soup.select('.nr-applet-nav-item.nr-list-link.openSubNav'):
    categories.append(category.attrs['href'].strip('/').split('/')[-1])
    print(category.attrs['href'])

https://tw.news.yahoo.com
https://tw.news.yahoo.com/politics
https://tw.news.yahoo.com/finance
https://tw.news.yahoo.com/entertainment
https://tw.news.yahoo.com/sports
https://tw.news.yahoo.com/society
https://tw.news.yahoo.com/local
https://tw.news.yahoo.com/world
https://tw.news.yahoo.com/lifestyle
https://tw.news.yahoo.com/health
https://tw.news.yahoo.com/technology
https://tw.news.yahoo.com/travel
https://tw.news.yahoo.com/weather/
https://tw.news.yahoo.com/odd
https://tw.news.yahoo.com/blogs
https://tw.news.yahoo.com/video
https://tw.news.yahoo.com/topic/Uk_Election
https://tw.news.yahoo.com/topic/Terrorist%20attack
https://tw.news.yahoo.com/topic/donaldtrump


### Remove unwanted categories

In [3]:
categories = categories[1:-6]
categories

['politics',
 'finance',
 'entertainment',
 'sports',
 'society',
 'local',
 'world',
 'lifestyle',
 'health',
 'technology',
 'travel',
 'weather']

### Get news list

In [4]:
req = requests.get('https://tw.news.yahoo.com/{}'.format(categories[0]))
soup = BeautifulSoup(req.text, 'lxml')

In [5]:
from urllib.parse import urljoin
urls = set()
for article in soup.select('.js-stream-content a'):
    url = urljoin('https://tw.news.yahoo.com/', article.attrs['href'])
    urls.add(url)
print(urls)

{'https://tw.news.yahoo.com/-015909701.html', 'https://tw.news.yahoo.com/%E4%BB%8A%E5%B9%B4%E6%96%B0%E8%88%88%E4%BA%9E%E6%B4%B2%E6%BC%B2%E5%B9%85%E5%B1%85%E5%86%A0-%E7%B6%93%E6%BF%9F%E6%95%B8%E6%93%9A%E6%9C%89%E6%92%90-%E5%8D%B0%E5%BA%A6%E8%82%A1%E5%B8%82%E4%B8%8D%E7%95%8F%E9%AB%98-015611093.html', 'https://tw.news.yahoo.com/%E5%BD%B1%E9%9F%B3-%E5%B9%B4%E9%87%91%E6%94%B9%E9%9D%A9%E6%B1%BA%E6%88%B0%E8%87%A8%E6%99%82%E6%9C%83-15%E6%97%A5%E6%8A%97%E7%88%AD%E5%86%8D%E8%B5%B7-012428362.html', 'https://tw.news.yahoo.com/%E4%BB%8A%E5%B9%B4%E6%96%B0%E8%88%88%E4%BA%9E%E6%B4%B2%E6%BC%B2%E5%B9%85%E5%B1%85%E5%86%A0-%E7%B6%93%E6%BF%9F%E6%95%B8%E6%93%9A%E6%9C%89%E6%92%90-%E5%8D%B0%E5%BA%A6%E8%82%A1%E5%B8%82%E4%B8%8D%E7%95%8F%E9%AB%98-015611648.html', 'https://tw.news.yahoo.com/%E9%9B%BB%E5%AD%90%E8%A3%9D%E7%BD%AE%E7%A6%81%E4%BB%A4-iata%E6%8F%90%E6%9B%BF%E4%BB%A3%E6%96%B9%E6%A1%88-012118565.html', 'https://tw.news.yahoo.com/%E5%B0%88%E8%A8%AA-%E5%8C%97%E5%B8%82%E5%8F%83%E9%81%B8%E7%88%86%E7%82%B8-%E4

### Pack it into a function

In [6]:
def get_news_links(category):
    page_url = 'https://tw.news.yahoo.com/{}'.format(category)
    req = requests.get(page_url)
    soup = BeautifulSoup(req.text, 'lxml')
    
    return {urljoin(page_url, article.attrs['href']) for article in soup.select('.js-stream-content a')}

links = {}
for category in categories:
    links[category] = get_news_links(category)

links.keys()

dict_keys(['politics', 'finance', 'entertainment', 'sports', 'society', 'local', 'world', 'lifestyle', 'health', 'technology', 'travel', 'weather'])

### Get news content

In [7]:
req = requests.get('https://tw.news.yahoo.com/%E6%94%BF%E9%99%A2-%E6%8E%A8%E5%8B%95%E5%B9%B4%E6%94%B9%E7%B5%95%E4%B8%8D%E5%8B%95%E6%90%96-160000837.html')
soup = BeautifulSoup(req.text, 'lxml')
soup.select_one('header.canvas-header h1').text

'政院：推動年改絕不動搖'

In [8]:
soup.select_one('div.auth-prov-soc').text

'【記者潘鴻志、王志誠／台北報導】台灣新生報2017年5月6日'

In [9]:
for p in soup.select('div.canvas-body p'):
    print(p.text)

行政院副院長林錫耀於昨（六）日代表林全院長出席「中華民國工商協進會中部工商午餐會」時表示，去（一○五）年五二○新政府上台後，積極解決各項問題，例如台灣面臨少子女化、高齡化等危機，因此，行政團隊積極推動年金改革，決心不會動搖，年金改革最重要的法案，已在立法院的委員會通過，盼在一個月內能完成二、三讀，也希望其它相關法案都能順利通過。
林副院長指出：除面對問題，推動改革外，政府也極力進行各項建設，林全院長今天就到台南訪視沙崙綠能科學城，總統也非常重視產業界的意見，充分聽取產業界的心聲，希望有效且積極解決問題，目前提出的八年八八○○億「前瞻基礎建設計畫」就是為了加速推動國家未來所需的建設，帶動經濟成長的動能，但「前瞻基礎建設計畫特別條例」目前還無法在立法院的委員會通過，盼各界給予支持，讓有利經濟民生的計劃儘速啟動。
林副院長並強調：新政府有決心，有計畫，堅定務實推動各項改革，並加速進行建設。台灣過去很長一段時間，經濟並不景氣。但新政府上台後，最近公布的一些經濟數據可發現，目前景氣已在穩定復甦中，政府希望在此時期，一方面加速各項基礎建設，擴大財政支出，帶動經濟成長；另外，也逐步推動各項經濟轉型政策，包括：「五＋二」創新產業計畫，期能透過產業創新帶動經濟轉型，讓台灣有更多元的經濟發展動能。
林副院長也指出：影響企業投資的障礙，例如水、電、人才、土地及環評等，政府都瞭解，也會積極溝通說明，希望取得更好的共識。


### Pack it into a function

In [10]:
def get_news_content(url):
    content = []
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'lxml')
    content.append(soup.select_one('header.canvas-header h1').text)
    content.append(soup.select_one('div.auth-prov-soc').text)
    for p in soup.select('div.canvas-body p'):
        content.append(p.text)

    return '\n'.join(content)

url = 'https://tw.news.yahoo.com/-043018999.html'
print(get_news_content(url))

八田與一銅像修復揭幕 長孫盼傳承到後世
政治中心／綜合報導民報2017年5月7日
台南烏山頭水庫八田與一銅像遭斷頭，日前完成修復，今（7）日上午舉行揭幕儀式。從日本趕來出席的八田與一長孫八田修一說，不敢相信這麼快就修復了，感到很開心，但他不是為銅像來的，而是來確認台日間的友誼沒變，而秀麗的風景沒變，景色依舊，他希望一切能傳承到後世。
台南市長賴清德則表示，八田與一銅像遭人蓄意破壞，市府未能妥善保護，對於家屬表達遺憾，但此種破壞行為不會影響台日感情，台日感情甚至會因為歷經考驗而更好。
八田與一銅像4月15日遭前台北市議員李承龍、邱姓女子斷首後，備受關注，連日本媒體也大幅報導；在頭部遍尋不著的情形下，王姓專業技師以奇美博物館典藏的胸像，接上銅像，完成修復。明天八田與一忌日，追思活動將如期舉行。
上午揭幕式包括台南市長賴清德、日本金澤市長山野之義、加賀市副市長河合篤史、日本台灣交流協會高雄事務所長中郡錦藏、日本金澤市訪問團，八田修一夫婦等家屬等多人參加，共同見證八田與一銅像的修復，並聲言台日友誼穩固。
金澤市長山野之義表示，這銅像是當初台灣朋友幫忙建造，台灣的朋友懷著感恩之心，幫我們故鄉的大前輩建造這銅像；感到開心的同時，身為台南市的友人他感到非常驕傲。
他表示，當然（銅像斷頭）這是一個令人不悅且傷心的事件，但台南與金澤、日本與台灣的友誼更深厚強固，他回到日本後，會把這好消息傳達給市民。
八田修一則說，看到祖父銅像在這麼短時間內修復，即使現場親眼目睹，他還是不敢相信；但他不是為銅像來的，而是要確認台日間的友誼沒變，不會因銅像而損及台日友誼。
八田修一表示，他的祖父75年前坐在此地，但這裡都沒有改變，水庫一樣在出水，風景依然秀麗，他希望能一直傳承到後世。
賴清德則表示，金澤市的傑出市民八田在八、九十年前東京帝大畢業後就來到台南，在物質相當缺乏的年代，建造了一個當時亞洲最現代化的水庫，不僅嘉慧農民，也促成了經濟發展。許許多多的農民、台灣的朋友，日本的好朋友們，大家都飲水思源，感念八田與一先生的偉大貢獻，集資興建銅像。
賴清德也表示，八田銅像4月26日就已修復完成，很有效率，沒有留下傷害的痕跡，非常感謝！這起令人遺憾的事件，目標是要破壞日本跟台灣的感情，慶幸的是台灣社會反對這樣的行為。他說，感情要經過考驗，考驗過的感情才是真感情，台日感情不僅沒變，還會更好。 
嘉南農田水利會會

### News ID parse function

In [11]:
import re
url_re = re.compile('-(\d{7,10})')

def parse_nid(url):
    m = url_re.search(url)
    
    return m.group(1)

parse_nid(url)

'043018999'

In [12]:
import os
for c in categories:
    os.mkdir(c)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'politics'

In [13]:
for category, urls in links.items():
    for url in urls:
        print(url)
        nid = parse_nid(url)
        filename = '{0}/{1}'.format(category, nid)
        if os.path.isfile(filename):
            continue
        content = get_news_content(url)
        with open(filename, 'w', encoding = 'utf-8') as f:
            print(content, file = f)

https://tw.news.yahoo.com/-015909701.html
https://tw.news.yahoo.com/%E4%BB%8A%E5%B9%B4%E6%96%B0%E8%88%88%E4%BA%9E%E6%B4%B2%E6%BC%B2%E5%B9%85%E5%B1%85%E5%86%A0-%E7%B6%93%E6%BF%9F%E6%95%B8%E6%93%9A%E6%9C%89%E6%92%90-%E5%8D%B0%E5%BA%A6%E8%82%A1%E5%B8%82%E4%B8%8D%E7%95%8F%E9%AB%98-015611093.html
https://tw.news.yahoo.com/%E5%BD%B1%E9%9F%B3-%E5%B9%B4%E9%87%91%E6%94%B9%E9%9D%A9%E6%B1%BA%E6%88%B0%E8%87%A8%E6%99%82%E6%9C%83-15%E6%97%A5%E6%8A%97%E7%88%AD%E5%86%8D%E8%B5%B7-012428362.html
https://tw.news.yahoo.com/%E4%BB%8A%E5%B9%B4%E6%96%B0%E8%88%88%E4%BA%9E%E6%B4%B2%E6%BC%B2%E5%B9%85%E5%B1%85%E5%86%A0-%E7%B6%93%E6%BF%9F%E6%95%B8%E6%93%9A%E6%9C%89%E6%92%90-%E5%8D%B0%E5%BA%A6%E8%82%A1%E5%B8%82%E4%B8%8D%E7%95%8F%E9%AB%98-015611648.html
https://tw.news.yahoo.com/%E9%9B%BB%E5%AD%90%E8%A3%9D%E7%BD%AE%E7%A6%81%E4%BB%A4-iata%E6%8F%90%E6%9B%BF%E4%BB%A3%E6%96%B9%E6%A1%88-012118565.html
https://tw.news.yahoo.com/%E5%B0%88%E8%A8%AA-%E5%8C%97%E5%B8%82%E5%8F%83%E9%81%B8%E7%88%86%E7%82%B8-%E4%B8%81%E5%AE%88%E

AttributeError: 'NoneType' object has no attribute 'text'